# Doraemon experiment

In [ ]:
from doraemon.doraemon import train_test_ppo_with_doraemon
import os

n_episodes = 20000
mean_timestep = 300
target_env = "CustomHopper-target-v0"
source_env = "CustomHopper-source-v0"

# optimized for PPO without UDR in source environment
optimized_clip_range = 0.19877024509129543
optimized_learning_rate = 0.0008
optimized_gamma = 0.992
seed=20

epsilon = 0.05
step = 2.0
delta = 0.5
alpha = 0.5

output_folder = f"doraemon-out/e{epsilon}-a{alpha}-d{delta}-s{step}"

os.mkdir(output_folder)

os.mkdir(f"{output_folder}/seed-{seed}")

print(f"PARAMETERS => eps: {epsilon}, step: {step}, delta: {delta}, alpha: {alpha}")
train_test_ppo_with_doraemon(
	output_folder=f"{output_folder}/seed-{seed}",
	train_env=source_env,
	test_env=target_env,
	episodes=n_episodes,
	clip_range=optimized_clip_range,
	learning_rate=optimized_learning_rate,
	gamma=optimized_gamma,
	timesteps=mean_timestep,
	print_std_deviation=True,
	seed=seed,
 	epsilon = epsilon,
	step = step,
	delta = delta,
	alpha = alpha
)

PARAMETERS => eps: 0.05, step: 2.0, delta: 0.5, alpha: 0.5


/opt/miniconda3/envs/mldl_m4/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean episode length
383.6182073903593
mean train reward
1276.8153005938505
mean test reward
992.2226002290035
test rewards file: doraemon-out/e0.05-a0.5-d0.5-s2.0/test_rewards_CustomHopper-target-v0.txt
mean test std
221.36658741131922
